In [1]:
%%javascript

Jupyter.keyboard_manager.command_shortcuts.add_shortcut('r', {
    help : 'run cell', 
    help_index : 'zz',
    handler : function (event) {
        
        IPython.notebook.execute_cell();
        return false;
    }}
);


<IPython.core.display.Javascript object>

In [2]:
# 속성 값에 접근하기
# __getattr__()와 _getattribute__() 메서드
# 인스턴스 객체에 대한 일반적인 접근 방법인 obj.attr() 메서드는 getattr(obj, 'attr')로 수행된다.
# 우선 __getattr__()과 _getattribute__() 의 차이.
# __getattr__() 메서드는 이름 공간에 정의되지 않는 이름에 접근할 때 호출되며 이에 대해서 처리 가능.

class GetAttr1(object):
    def __getattr__(self, x):
        print('__getattr__', x)
        if x == 'test':
            return 10
        raise AttributeError('Not good')
        
g1 = GetAttr1()
g1.c = 10
print(g1.c)
# print(g1.a) # __getattr__() 메서드가 호출된다.
print(g1.test) # 정의되지 않았지만 준비된 이름.
# 이 예제로 안 것. 그러니까 __getattr__() 은 객체 안, 즉 객체 이름공간 안에 정의 되지 않는 이름에 접근 하려고 하면 호출 됨.

10
__getattr__ test
10


In [5]:
# __getattribute__() 메서드는 이름 정의 여부에 관계없이 모든 속성에 접근하면 호출된다.
class GetAttr2(object):
    def __getattribute__(self, x):
        print('__getattribute__ called', x)
        return object.__getattribute__(self, x)
    
g2 = GetAttr2()
g2.c = 10
print(g2.c)
print(g2.a) # __getattr__()와는 다르게 정의되지 않은 이름도 호출한다. 
# 따라서 __getattribute__()메서드는 호출되는 이름 전체에 대한 제어권을 얻어 낸다.

__getattribute__ called c
10
__getattribute__ called a


AttributeError: 'GetAttr2' object has no attribute 'a'

In [ ]:
# 두 메서드가 모두 정의 되어 있는 예
# 정의되어 있는 이름에 접근할 때는 __getattribute__() 메서드를 호출하고, 정의되어 있지 않은 이름에 접근할 때는
# __getattribute__()와 __getattr__() 메서드 모두를 호출한다.

class GetAttr3(object):
    def __getattr__(self, x):
        print('__getattr__', x)
        raise AttributeError
    def __getattribute__(self, x):
        print('__getattribute__ called..', x)
        return object.__getattribute__(self, x)
    
g3 = GetAttr3()
g3.c = 10
print(g3.c) # 정의된 속성에 접근 __getattribute__()만 호출
print(g3.a) # 정의되어 있지 않은 속성에 접근 __getattr__(), __getattribute__() 둘다 호출

# 그니까 걍 어떤 객체에 접근하려고 하면 커맨드 창에서 그 객체를 타이핑하면 나올 것임.
# 이 때 타이핑하면 그 객체의 속성이 나오는데 이때 함수가 __getattr__, __getattribute__ 메서드.
# 객체에 접근했을 때, 단순히 접근만 했을 때 돌려주는 속성을 정해주는 파트.
# 주의할 점은, self.__getattribute__(x)와 같이 호출하면 재귀적으로 자기 자신을 무한히 호출함. 
# 그러므로 상위 클래스를 통해서 object._getattribute__(self, x)와 같은 식으로 접근해야 함.

In [9]:
# __setattr__()와 __delattr__() 메서드
# 인스턴스 객체에서 속성을 설정할 때는 __setattr__()메서드를, 속성을 삭제할 때는 __delattr__메서드를 사용.
# obj.x = o 는 setattr(obj, 'x', o)로 수행되며 obj.__setattr__('x', o)를 호출하고 
# del obj.x 는 delattr(obj, 'x')로 수행되며 obj.__delattr__('x')를 호출한다.

class Attr:
    def __setattr__(self, name, value):
        print('__setattr__(%s) = %s called' % (name, value))
        object.__setattr__(self, name, value)
    def __delattr__(self, name):
        print('__delattr__(%s) called' % name)
        object.__delattr__(self, name)
a = Attr()
a.x = 10
print(a.x)
del a.x
print(a.x)
                    

__setattr__(x) = 10 called
10
__delattr__(x) called


AttributeError: 'Attr' object has no attribute 'x'

In [24]:
# 인스턴스 객체를 호출하기 : __call__() 메서드
# 어떤 클래스 인스턴스가 __call__() 메서드를 가지고 있으면, 해당 인스턴스 객체는 함수와 같은 모양으로 호출할 수 있다.
# 인스턴스 객체 x에 대해 다음과 같이 확장된다. x(a1, a2, a3) > x.__call__(a1, a2, a3)

# 다음 클래스 Factorial 은 고속 처리를 위하여 기억 기법 사용. 한 번 계산된 팩토리얼 값은 인스턴스 객체의 cache 멤버에
# 저장되어 있다가 필요할 때 다시 사용한다. 팩토리얼 계산은 인스턴스 객체의 __call__() 메서드를 호출하여 이루어진다.
# 이 예제 좋네. __call__ 메서드를 가진 객체는 함수처럼 인수를 전달하여 호출할 수 있어서 함수와 비슷하게 동작.
# 다시 계산하면 비효율적이니, cache를 두어 (기억 기법), 한번 계산하고 난 다음에는 바로 참고만 하도록 함.
class Factorial:
    def __init__(self):
        self.cache = {}
    def __call__(self, n):
        if n not in self.cache:
            if n == 0:
                self.cache[n] = 1
            else:
                self.cache[n] = n * self.__call__(n-1)
        return self.cache[n]
fact = Factorial()
for i in range(10):
    print('{}! = {}'.format(i, fact(i)))
    
fact.cache.items()

0! = 1
1! = 1
2! = 2
3! = 6
4! = 24
5! = 120
6! = 720
7! = 5040
8! = 40320
9! = 362880


dict_items([(0, 1), (1, 1), (2, 2), (3, 6), (4, 24), (5, 120), (6, 720), (7, 5040), (8, 40320), (9, 362880)])

In [29]:
# 호출 가능한지 확인하기
# 어떤 객체가 호출 가능한지 알아보려면 collections.Callable의 인스턴스 객체인지 확인한다.
import collections
def f():
    pass
print(isinstance(f, collections.Callable))
fact = Factorial()
print(isinstance(fact, collections.Callable))

True
True
